In [2]:
## IMPORTS
import pandas as pd
import numpy as np
from datetime import date,datetime, timedelta
import yahoo_fin.stock_info as yf

In [41]:
# SectorDict = {
#     "Energy":"XLE",
#     "Technology":"XLK",
#     "Communication Services":"XLC",
#     "Consumer Defensive":"XLP",
#     "Healthcare":"XLV",
#     "Consumer Cyclical":"XLY",
#     "Industrials":"XLI",
#     "Utilities":"XLU",
#     "Basic Materials":"XLB",
#     "Financial Services":"XLF",
#     "Real Estate":"XLRE"}
# spList = yf.tickers_sp500()
# today = date.today()
# today = today.strftime('%Y-%m-%d')
dictionary = {}
# test = ['AAPL']
# for ticker in spList:
#     try:
#         # df = yf.get_data(ticker, start_date="1995-01-02",end_date="2022-01-09", index_as_date = True, interval="1wk")
#         # df.to_pickle('stockData/S&P500/'+ticker+'.pkl')
#         print(ticker)
#         industry = yf.get_company_info(ticker).loc['sector'][0]
#         dictionary[ticker] = SectorDict[industry]
#     except:
#         print("stock not found")

In [42]:
# print(dictionary)

In [43]:
import pickle
try:
    sectorDict = open('stockData/sector.pkl', 'wb')
    pickle.dump(dictionary, sectorDict)
    sectorDict.close()
  
except:
    print("Something went wrong")

In [3]:
import yahoo_fin.stock_info as yf
import numpy as np
import pandas as pd
SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
now = datetime.now()
sectorDfList = []
for sector in SectorDict.values():
    df = yf.get_data(sector, start_date="1995-01-06",end_date = now, index_as_date = True)
    df['Dates'] = pd.to_datetime(df.index)
    df = df[df['Dates'].dt.weekday == 4]
    df = df.drop('Dates', axis = 1)
    sectorDfList.append(df)
    sectorDfList[-1].to_pickle("stockData/sectorCharts/"+sector+".pkl")

In [4]:
weights = np.arange(1,31)
sumWeights = np.sum(weights)
spDf = yf.get_data('^GSPC', start_date="1995-01-06",end_date = now, index_as_date = True)
spDf['Dates'] = pd.to_datetime(spDf.index)
spDf = spDf[spDf['Dates'].dt.weekday == 4]
spDf = spDf.drop('Dates', axis = 1)
def product(df,index):
    if index < 30:
        return 0
    else:
        return df.iloc[index].close/df.iloc[index-30].close
def calculateSlope(df,index):
    if index < 31:
        return 0
    else:
        return df.iloc[index]['MA30']/df.iloc[index-1]['MA30']
for df in sectorDfList:
    df['MA30'] = df['close'].rolling(window=30).apply(lambda x: np.sum(x)/30)
    df['MA30Slope'] = df.apply(lambda x: calculateSlope(df,df.index.get_loc(x.name)),axis=1)
    df['percent'] = df.apply(lambda x: product(df,df.index.get_loc(x.name)),axis=1)
    spDf['percent'] = spDf.apply(lambda x: product(spDf,spDf.index.get_loc(x.name)),axis=1)
    df['RS'] = df['percent'] - spDf['percent']
    df.drop(['open','high','low','adjclose','volume','percent'],axis=1)
    df.to_pickle("stockData/sectorCharts/"+df.iloc[-1]['ticker']+".pkl")

In [16]:
fullPrint(pd.read_pickle("stockData/sectorCharts/XLC.pkl"))
fullPrint(spDf)

                 open       high        low      close   adjclose    volume  \
2018-06-22  50.590000  50.610001  50.189999  50.490002  48.974186    181500   
2018-06-29  49.799999  49.840000  49.419998  49.529999  48.043007    152300   
2018-07-06  50.139999  50.816002  49.930000  50.770000  49.245777    199800   
2018-07-13  51.779999  51.799999  51.449001  51.689999  50.138157     80100   
2018-07-20  51.049999  51.186001  50.840000  50.869999  49.342781    158400   
2018-07-27  49.770000  49.830002  48.638000  48.910000  47.441624   1111200   
2018-08-03  48.090000  48.930000  48.090000  48.910000  47.441624    117200   
2018-08-10  49.529999  49.529999  49.150002  49.270000  47.790810    177600   
2018-08-17  48.660000  48.810001  48.400002  48.610001  47.150623    226900   
2018-08-24  48.959999  49.439999  48.880001  49.439999  47.955708    330900   
2018-08-31  49.619999  49.619999  49.042000  49.180000  47.703522    235000   
2018-09-07  47.279999  47.950001  47.230000  47.6800

In [5]:
goodSectorDf = sectorDfList[0].drop(['close','ticker','MA30','MA30Slope','RS'],axis=1)

In [6]:
from stage import fullPrint
goodSectorDf.index = sectorDfList[0].index
fullPrint(sectorDfList[1][['MA30Slope','RS']])

            MA30Slope        RS
1999-01-08   0.000000 -1.148244
1999-01-15   0.000000 -1.139748
1999-01-22   0.000000 -1.099950
1999-01-29   0.000000 -1.164537
1999-02-05   0.000000 -1.126062
1999-02-12   0.000000 -1.085537
1999-02-19   0.000000 -1.064320
1999-02-26   0.000000 -1.043463
1999-03-05   0.000000 -1.118049
1999-03-12   0.000000 -1.155193
1999-03-19   0.000000 -1.192611
1999-03-26   0.000000 -1.207057
1999-04-09   0.000000 -1.247040
1999-04-16   0.000000 -1.284148
1999-04-23   0.000000 -1.393227
1999-04-30   0.000000 -1.323192
1999-05-07   0.000000 -1.318511
1999-05-14   0.000000 -1.280498
1999-05-21   0.000000 -1.326840
1999-05-28   0.000000 -1.322484
1999-06-04   0.000000 -1.256839
1999-06-11   0.000000 -1.208253
1999-06-18   0.000000 -1.222241
1999-06-25   0.000000 -1.152759
1999-07-02   0.000000 -1.235849
1999-07-09   0.000000 -1.206033
1999-07-16   0.000000 -1.189922
1999-07-23   0.000000 -1.153135
1999-07-30   0.000000 -1.139105
1999-08-06   0.000000 -1.094493
1999-08-

In [7]:
goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
goodSectorDf['BadSectors'] = [[] for _ in range(len(goodSectorDf))]
for date in sectorDfList[0].index:
    listOfSector = []
    listOfBadSector = []
    for df in sectorDfList:
        try:
            if df.at[date,'MA30Slope'] > 1 or df.loc[date,'RS'] > 0.1:
                if df.at[date,'close'] > df.at[date,'MA30']:
                    listOfSector.append(df.at[date,'ticker'])
            if (df.at[date,'MA30Slope'] < 0.995 and df.loc[date,'RS'] < 0) or df.at[date, 'MA30Slope'] < 0.975:
                if df.at[date,'close'] < df.at[date,'MA30']:
                    listOfBadSector.append(df.at[date,'ticker'])
        except: 
            print(df.iloc[-1]['ticker'])
    goodSectorDf.at[date,'Sectors'] = set(listOfSector)
    goodSectorDf.at[date,'BadSectors'] = set(listOfBadSector)

XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
X

In [8]:
from stage import fullPrint
fullPrint(goodSectorDf)
goodSectorDf.to_pickle('stockData/goodSector.pkl')

                  open        high         low   adjclose    volume   percent  \
1999-01-08   24.000000   24.562500   23.375000  13.730424    421100  0.000000   
1999-01-15   22.562500   22.750000   22.453125  13.014358     10700  0.000000   
1999-01-22   22.750000   22.812500   22.562500  12.942754     19300  0.000000   
1999-01-29   21.687500   21.812500   21.437500  12.495217     18600  0.000000   
1999-02-05   22.312500   22.937500   22.187500  13.103869     66000  0.000000   
1999-02-12   22.375000   22.625000   22.156250  12.692130    242300  0.000000   
1999-02-19   21.937500   21.968750   21.531250  12.441515    130300  0.000000   
1999-02-26   21.843750   22.000000   21.625000  12.387813     24900  0.000000   
1999-03-05   23.000000   23.031250   22.625000  13.094914     92700  0.000000   
1999-03-12   24.875000   25.062500   24.250000  14.034743    604300  0.000000   
1999-03-19   25.250000   25.281250   24.625000  14.246896    260400  0.000000   
1999-03-26   24.687500   24.